In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [2]:
# Load the dataset
file_path = "C:/Users/Ganes/OneDrive/salestable.csv"
df = pd.read_csv(file_path)

# Rename column if necessary
df.rename(columns={'qty_orderded': 'qty_ordered'}, inplace=True)

# Convert 'order_date' to datetime format
df['order_date'] = pd.to_datetime(df['order_date'])

# Extract year and month
df['year_month'] = df['order_date'].dt.to_period('M')

# Aggregate total sales quantity per month
monthly_sales = df.groupby('year_month')['qty_ordered'].sum().reset_index()

# Convert 'year_month' to datetime for modeling
monthly_sales['year_month'] = monthly_sales['year_month'].astype(str)
monthly_sales['year_month'] = pd.to_datetime(monthly_sales['year_month'])

# Compute total sales per quarter
monthly_sales['quarter'] = monthly_sales['year_month'].dt.to_period('Q')
quarterly_sales = monthly_sales.groupby('quarter')['qty_ordered'].sum().reset_index()

# Print total sales per quarter
print("\nTotal Sales per Quarter:")
print(quarterly_sales)


Total Sales per Quarter:
  quarter  qty_ordered
0  2021Q1          715
1  2021Q2          529
2  2021Q3         1738
3  2021Q4         2285
4  2022Q1         2240
5  2022Q2         2534
6  2022Q3         2986
7  2022Q4          501


In [5]:
# Filter data for 2022 (training data)
train_data = monthly_sales[(monthly_sales['year_month'] >= '2022-01-01') & 
                           (monthly_sales['year_month'] <= '2022-12-01')]

# Fit an ARIMA model (AutoRegressive Integrated Moving Average)
model = ARIMA(train_data['qty_ordered'], order=(2,1,2))  # Using ARIMA(2,1,2) as an initial guess
model_fit = model.fit()

# Forecast for Q2 2023 (April, May, June)
forecast_steps = 3
forecast = model_fit.forecast(steps=forecast_steps)

# Create a date range for Q2 2023
forecast_dates = pd.date_range(start='2023-04-01', periods=forecast_steps, freq='MS')

# Combine results into a DataFrame
forecast_df = pd.DataFrame({'year_month': forecast_dates, 'predicted_qty_ordered': forecast})

# Model Evaluation
actual_data = monthly_sales[(monthly_sales['year_month'] >= '2022-10-01') &
                            (monthly_sales['year_month'] <= '2022-12-01')]

mae = mean_absolute_error(actual_data['qty_ordered'], model_fit.fittedvalues[-3:])
rmse = np.sqrt(mean_squared_error(actual_data['qty_ordered'], model_fit.fittedvalues[-3:]))

# Print results
print("\nPredictions for Q2 2023:")
print(forecast_df)
print(f"\nModel Evaluation:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")

c:\Users\Ganes\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\Ganes\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\Ganes\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\Ganes\AppData\Local\Programs\Python\Python312\Lib


Predictions for Q2 2023:
   year_month  predicted_qty_ordered
12 2023-04-01             407.742889
13 2023-05-01             602.260106
14 2023-06-01             608.009291

Model Evaluation:
MAE: 401.54
RMSE: 429.75
